In [1]:
import open_clip
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image
import torch.nn.functional as F
import matplotlib.pyplot as plt

# 1. Încarcă modelul și tokenizer-ul CLIP
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
    'hf-hub:laion/CLIP-ViT-B-32-laion2B-s34B-b79K'
)
tokenizer = open_clip.get_tokenizer('hf-hub:laion/CLIP-ViT-B-32-laion2B-s34B-b79K')

# 2. Definirea transformărilor pentru preprocesarea imaginii
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Transformă imaginea în RGB (3 canale)
    transforms.Resize((224, 224)),  # Redimensionează la dimensiunea necesară pentru CLIP
    transforms.ToTensor(),          # Convertește imaginea într-un tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalizarea imaginii
])

# 3. Încarcă dataset-ul FashionMNIST
def load_fashion_mnist():
    # Încarcă setul de date FashionMNIST (train sau test)
    dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    return dataloader

# 4. Crează embedding pentru text
def process_text(text):
    # Tokenizează și creează embedding pentru text
    text_input = tokenizer([text])
    text_features = model.encode_text(text_input)
    return text_features

# 5. Compară imaginea și textul folosind similaritatea cosine
def compare_image_and_text(image_features, text_features):
    # Calculăm similaritatea cosine între embedding-urile imaginii și textului
    similarity = F.cosine_similarity(image_features, text_features)
    return similarity.item()

# 6. Aplica modelul pe un set de imagini și compară cu descrierea textului
def apply_model_on_fashion_mnist(model, dataloader, description, max_images=100):
    results = []
    image_count = 0  # Contor pentru a limita procesarea la 100 de imagini

    # Creăm embedding pentru descrierea textului
    text_features = process_text(description)

    for images, labels in dataloader:
        if image_count >= max_images:  # Oprește procesarea după 100 de imagini
            break

        # Procesează fiecare imagine din batch
        for i in range(images.size(0)):
            if image_count >= max_images:  # Verifică dacă am procesat deja 100 de imagini
                break

            image = images[i].unsqueeze(0)  # Adaugă o dimensiune batch

            with torch.no_grad():  # Evită calculul gradientului pentru inferență
                # Obține embedding-ul imaginii
                image_features = model.encode_image(image)

                # Compară imaginea cu textul
                similarity = compare_image_and_text(image_features, text_features)

                results.append({
                    'image': labels[i].item(),  # Eticheta (de exemplu, 0 pentru T-shirt, 1 pentru trouser etc.)
                    'similarity': similarity,
                    'image_tensor': images[i]  # Salvează tensorul imaginii pentru vizualizare
                })

            image_count += 1  # Crește contorul de imagini procesate

    return results

# 7. Vizualizarea imaginilor
def show_image(image_tensor):
    # Convertește tensorul în imagine folosind matplotlib
    image = image_tensor.permute(1, 2, 0)  # Permută dimensiunile pentru a fi compatibile cu matplotlib
    image = image.numpy()  # Convertește tensorul într-un array numpy
    plt.imshow(image)  # Afișează imaginea
    plt.axis('off')  # Ascunde axele
    plt.show()

# 8. Testează modelul pe setul de imagini FashionMNIST cu o descriere
if __name__ == "__main__":
    # Încarcă setul de date FashionMNIST
    dataloader = load_fashion_mnist()

    # Descrierea textului de căutat
    description = "T-shirt"  # Schimbă descrierea în funcție de testul dorit

    # Aplică modelul pe setul de imagini FashionMNIST și compară cu descrierea
    results = apply_model_on_fashion_mnist(model, dataloader, description)

    # Afișează primele 10 rezultate și vizualizează imaginile
    for result in results[:10]:  # Afișăm primele 10 rezultate
        print(f"Eticheta imaginei: {result['image']} - Similaritate cu '{description}': {result['similarity']}")
        # Vizualizează imaginea
        show_image(result['image_tensor'])


ModuleNotFoundError: No module named 'open_clip'